In [16]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

fname='./brazil_cities.csv'

df=pd.read_csv(fname)

cities={}

for index, row in df.iterrows():
    city_name=row['URL'].split('/')[-1]
    
    cities[city_name]=[
        float(row['latitude'].replace(',', '.')), 
        float(row['longitude'].replace(',', '.'))
    ]

In [17]:
cities

{'Assis_Brasil': [-10.9408333333333, -69.5669444444444],
 'Brasil%C3%A9ia': [-11.0, -68.7333333333333],
 'Epitaciol%C3%A2ndia': [-11.0288888888889, -68.7408333333333],
 'Xapuri': [-10.6519444444444, -68.5038888888889],
 'Acrel%C3%A2ndia': [-9.82777777777778, -66.8827777777778],
 'Bujari': [-9.8214, -67.9564],
 'Capixaba_Acre': [-10.5727777777778, -67.6758333333333],
 'Pl%C3%A1cido_de_Castro_Acre': [-10.2758333333333, -67.15],
 'Porto_Acre': [-9.58777777777778, -67.5327777777778],
 'Rio_Branco_Acre': [-9.97472222222222, -67.81],
 'Senador_Guiomard': [-10.1508333333333, -67.7358333333333],
 'Manoel_Urbano': [-8.8325, -69.2694],
 'Santa_Rosa_do_Purus': [-9.43277777777778, -70.4927777777778],
 'Sena_Madureira': [-9.06583333333333, -68.6569444444445],
 'Cruzeiro_do_Sul_Acre': [-7.63083333333333, -72.67],
 'M%C3%A2ncio_Lima': [-7.61388888888889, -72.8958333333333],
 'Marechal_Thaumaturgo': [-8.94083333333333, -72.7919444444444],
 'Porto_Walter': [-8.26888888888889, -72.7438888888889],
 'Rodr

In [18]:
cities['Mazag%C3%A3o']

[-0.115, -51.2888888888889]

In [21]:
list(dist_lst)

[]

In [25]:
[city_item for city_item in list(cities.items()) if city_item[0] != city_name]

[('Brasil%C3%A9ia', [-11.0, -68.7333333333333]),
 ('Epitaciol%C3%A2ndia', [-11.0288888888889, -68.7408333333333]),
 ('Xapuri', [-10.6519444444444, -68.5038888888889]),
 ('Acrel%C3%A2ndia', [-9.82777777777778, -66.8827777777778]),
 ('Bujari', [-9.8214, -67.9564]),
 ('Capixaba_Acre', [-10.5727777777778, -67.6758333333333]),
 ('Pl%C3%A1cido_de_Castro_Acre', [-10.2758333333333, -67.15]),
 ('Porto_Acre', [-9.58777777777778, -67.5327777777778]),
 ('Rio_Branco_Acre', [-9.97472222222222, -67.81]),
 ('Senador_Guiomard', [-10.1508333333333, -67.7358333333333]),
 ('Manoel_Urbano', [-8.8325, -69.2694]),
 ('Santa_Rosa_do_Purus', [-9.43277777777778, -70.4927777777778]),
 ('Sena_Madureira', [-9.06583333333333, -68.6569444444445]),
 ('Cruzeiro_do_Sul_Acre', [-7.63083333333333, -72.67]),
 ('M%C3%A2ncio_Lima', [-7.61388888888889, -72.8958333333333]),
 ('Marechal_Thaumaturgo', [-8.94083333333333, -72.7919444444444]),
 ('Porto_Walter', [-8.26888888888889, -72.7438888888889]),
 ('Rodrigues_Alves_Acre', [-7

In [38]:
dist_lst=list(map(dist_tuple_fun, [city_item for city_item in list(cities.items()) if city_item[0] != city_name]))
dist_lst.sort(key=lambda x: x[1])
dist_lst=dist_lst[:k]
dist_lst

[('Epitaciol%C3%A2ndia', 90.70625246980073),
 ('Brasil%C3%A9ia', 91.23688936983854),
 ('Xapuri', 120.47533445552777)]

In [46]:
from spycio import distance
from tqdm import tqdm

k=10
radius=6371
config={'radius':radius}

dist_fun=lambda A, B: distance(A, B, 'geographical', config)

closest_cities={}

for city_name in tqdm(cities.keys()):
    city_position=cities[city_name]
    
    dist_tuple_fun=lambda candidate: (candidate[0], dist_fun(city_position, candidate[1]))
    dist_lst=list(
        map(
            dist_tuple_fun, 
            [city_item for city_item in list(cities.items()) if city_item[0] != city_name]
        )
    )
    
    dist_lst.sort(key=lambda x: x[1])
    
    closest_cities[city_name]=dist_lst[:k]

100%|███████████████████████████████████████| 5554/5554 [21:13<00:00,  4.36it/s]


In [44]:
dist_fun(cities['Itau%C3%A7u'], cities['Inhumas'])

20.46451941262333

In [51]:
import json

# Serializing json
json_object = json.dumps(closest_cities, indent=4)

fname="{k}_closest_cities.json".format(k=k)

# Writing to sample.json
with open(fname, "w") as outfile:
    outfile.write(json_object)